In [1]:
import os
import sys
sys.path.append(os.environ["PROBLOG_HOME"])
import problog
from problog.tasks import sample
from problog.program import PrologString

In [2]:
def save_file(path, content):
    try:
        os.remove(path)
    except OSError:
        pass
    with open(path, 'a') as out:
        out.write(content+'\n')

In [3]:
def read_lines(file_path):
    array = []
    with open(file_path, "r") as ins:
        for line in ins:
            line = line.replace("\n","").replace(" ","")
            if len(line)>0:
                array.append(line)
    return array

In [4]:
def get_list(edge_path, user_path):
    edges_initial = read_lines(edge_path)
    edges = []
    for e in edges_initial:
        edge = e.split("\t")
        #print(edge)
        if len(edge)>1:
            edges.append((edge[0], edge[1]))    
    users = read_lines(user_path)
    return edges, users

In [5]:
modeltext = """
    trusts_directed(bernd,ingo).
    trusts_directed(ingo,theo).
    trusts_directed(theo,angelika).
    trusts_directed(bernd,martijn).
    trusts_directed(ingo,martijn).
    trusts_directed(martijn,guy).
    trusts_directed(guy,theo).
    trusts_directed(guy,angelika).
    trusts_directed(laura,ingo).
    trusts_directed(laura,theo).
    trusts_directed(laura,guy).
    trusts_directed(laura,martijn).
    trusts_directed(kurt,bernd).
    
    0.4:: trusts(X,Y) :- trusts_directed(Y,X).
    0.4:: trusts(X,Y) :- trusts_directed(X,Y).
    0.3:: trusts(X,Y) :- trusts_directed(X,Z), trusts(Z,Y).
    
    query(trusts(bernd,ingo)).
    query(trusts(ingo,theo)).
    query(trusts(laura,angelika)).
    query(trusts(guy,martijn)).
    query(trusts(ingo,martijn)).
    query(trusts(martijn,bernd)).
    query(trusts(bernd,theo)).
    query(trusts(bernd,theo)).
    query(trusts(bernd,theo)).
    query(trusts(bernd,theo)).
    query(trusts(bernd,golnoosh)).
"""

In [6]:
def get_samples(modeltext, sample_size):
    model = PrologString(modeltext)
    result = sample.sample(model, n=sample_size, format='dict')
    return result   

In [7]:
result = get_samples(modeltext, sample_size=3)

In [8]:
for i in result:
    print(i)

{trusts(martijn,bernd): False, trusts(bernd,golnoosh): False, trusts(guy,martijn): False, trusts(ingo,martijn): True, trusts(laura,angelika): False, trusts(ingo,theo): False, trusts(bernd,theo): False, trusts(bernd,ingo): True}
{trusts(martijn,bernd): False, trusts(bernd,golnoosh): False, trusts(guy,martijn): True, trusts(ingo,martijn): False, trusts(laura,angelika): False, trusts(ingo,theo): True, trusts(bernd,theo): True, trusts(bernd,ingo): False}
{trusts(martijn,bernd): True, trusts(bernd,golnoosh): False, trusts(guy,martijn): False, trusts(ingo,martijn): True, trusts(laura,angelika): False, trusts(ingo,theo): False, trusts(bernd,theo): False, trusts(bernd,ingo): True}


In [9]:
def generate_model_text(sample_graph_path,node_size):
    model_text = ""
    model = """
0.4:: trusts(X,Y) :- trusts_directed(Y,X).
0.4:: trusts(X,Y) :- trusts_directed(X,Y).
0.3:: trusts(X,Y) :- trusts_directed(X,Z), trusts(Z,Y).
    """
    trust_file = sample_graph_path+"trust-"+str(node_size)+".txt"
    user_file = sample_graph_path+"user-"+str(node_size)+".txt"
    trusts, users = get_list(trust_file,user_file)
    for trust in trusts:
        model_text+="trusts_directed("+trust[0]+","+trust[1]+")."+'\n'
    model_text+=model+'\n'
    query_text = ""
    for user1 in users:
        for user2 in users:
            if user1!= user2:
                query_text+="query(trusts("+user1+","+user2+"))."+'\n'
    model_text+= query_text+"\n" 
    model_path = sample_graph_path+"problog_model-"+str(node_size)+".txt"
    save_file(model_path, model_text)
    return model_text

In [95]:
sample_graph_path = "../sample_graphs/"
model_text = generate_model_text(sample_graph_path,node_size=10)

result = get_samples(model_text, sample_size=3)
for i in result:
    print(i)

{trusts(0,8): True, trusts(9,3): False, trusts(7,6): True, trusts(8,3): False, trusts(0,6): True, trusts(2,5): True, trusts(4,7): True, trusts(1,9): False, trusts(8,0): True, trusts(0,2): True, trusts(9,8): False, trusts(6,5): True, trusts(5,4): True, trusts(6,4): True, trusts(5,9): False, trusts(2,6): True, trusts(3,0): True, trusts(8,6): True, trusts(0,5): False, trusts(9,5): True, trusts(2,0): False, trusts(5,7): False, trusts(5,8): False, trusts(5,1): False, trusts(1,3): False, trusts(8,9): False, trusts(0,9): True, trusts(1,7): True, trusts(3,4): False, trusts(7,1): True, trusts(4,1): False, trusts(6,7): False, trusts(2,3): False, trusts(2,8): True, trusts(6,9): True, trusts(1,2): False, trusts(1,0): False, trusts(2,1): True, trusts(0,4): False, trusts(0,3): False, trusts(7,0): True, trusts(4,2): False, trusts(6,0): False, trusts(7,3): False, trusts(6,2): True, trusts(5,0): False, trusts(6,1): True, trusts(7,4): True, trusts(5,6): True, trusts(4,5): True, trusts(5,2): False, trust

In [10]:
import pickle
def save_to_pickle(dict_to_save, path_to_save):
    with open(path_to_save, 'wb') as handle:
        pickle.dump(dict_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def read_pickle(pickle_path):
    b= {}
    with open(pickle_path, 'rb') as handle:
        b = pickle.load(handle)
    return b

In [11]:
def make_dictionary (sample_result):
    dict_to_save = {}
    for sample_dict in sample_result:
        for item in sample_dict.keys():
            if item in dict_to_save.keys():
                samples_of_item = dict_to_save[item]
                samples_of_item.append(sample_dict[item])
                dict_to_save[item] = samples_of_item
            else:
                dict_to_save[item] = [sample_dict[item]]
    return dict_to_save

In [12]:
def save_samples(sample_graph_path, node_size, sample_size):
    model_text = generate_model_text(sample_graph_path,node_size)
    result = get_samples(model_text, sample_size)
    path_to_save = sample_graph_path+"generated_sample_dict-"+str(node_size)+"("+str(sample_size)+")"+".pickle"
    dict_to_save = make_dictionary (result)
    save_to_pickle(dict_to_save, path_to_save)

In [14]:
sample_graph_path = "../sample_graphs/"
save_samples(sample_graph_path, node_size = 14, sample_size = 1000)

In [110]:
sample_graph_path = "../sample_graphs/"
pickle_path = sample_graph_path+"generated_sample_dict-"+str(10)+"("+str(10)+")"+".pickle"
test_dict = read_pickle(pickle_path)

In [111]:
test_dict

{trusts(0,8): [False,
  False,
  True,
  False,
  True,
  False,
  False,
  False,
  False,
  False],
 trusts(9,2): [False, True, True, False, True, True, True, True, False, True],
 trusts(7,6): [True,
  False,
  False,
  True,
  False,
  False,
  True,
  True,
  False,
  False],
 trusts(8,6): [False,
  False,
  False,
  True,
  False,
  True,
  False,
  True,
  False,
  False],
 trusts(0,3): [False,
  False,
  False,
  True,
  False,
  True,
  True,
  True,
  True,
  False],
 trusts(2,1): [True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  True],
 trusts(4,3): [False,
  False,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False],
 trusts(1,9): [False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  True],
 trusts(8,2): [True,
  False,
  False,
  True,
  False,
  False,
  False,
  True,
  True,
  False],
 trusts(0,5): [False,
  False,
  True,
  False,
  False,
  False,
  False,
  True,
  False,
  True],
 trusts(6,9)